In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn import metrics
from torch.autograd import Variable
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data preparation 

In [ ]:
%cd /content/drive/MyDrive/MNSIT

/content/drive/MyDrive/MNSIT


In [ ]:
%pwd

'/content/drive/MyDrive/MNSIT'

In [ ]:
!unzip archive.zip

In [ ]:
os.chdir("trainingSet")

In [ ]:
data = []
labels = []
ls=os.listdir()
for dir in ls:
  os.chdir(dir)
  currentDir = os.getcwd()
  images = os.listdir()
  paths = [os.path.abspath(path) for path in images ]
  data = data + paths
  labels = labels +  len(paths)*[dir]
  os.chdir("..")





# split data (train, val, test)

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1 , stratify = labels)

# data loader

In [ ]:
image = cv2.imread("/content/drive/MyDrive/MNSIT/trainingSet/trainingSet/0/img_1.jpg",0)
image.shape
image=image.reshape((1,28,28))
image.shape

(1, 28, 28)

In [ ]:
class myDataset(Dataset):
  def __init__(self, array,label):
    self.array = array
    self.label = label
      # stuff
      
  def __getitem__(self, index):
      # stuff
      image = cv2.imread(self.array[index],0)
      image=image.reshape((1,28,28))
        
      return torch.tensor(image).float(), torch.tensor(int(self.label[index]))

  def __len__(self):
      return len(self.array) # of how many examples(images?) you have

In [ ]:
customDataset=myDataset(X_train,y_train)

In [ ]:
train_dataloader = DataLoader(customDataset, batch_size=4,
                        shuffle=True, num_workers=0)

#Model

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

training loop

In [ ]:
def run_model(model,dataloader, optimizer,train = True ):
  if train:
    model.train()
  
  pred = []
  labels = []
  loss = nn.CrossEntropyLoss()
  total_loss = 0
  for (data, label) in dataloader:
       
        optimizer.zero_grad()
        output = model(data)
        loss_ = loss(output, label)
        total_loss += loss_.item()
        loss_.backward()
        optimizer.step()
        pred += output.tolist()
        labels += label.tolist()
        # print(total_loss)

  return labels,pred,total_loss/len(dataloader)





In [ ]:
model=Model()
epoch = 20

optimizer = torch.optim.Adam(model.parameters(), 0.00001, weight_decay=.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=.3, threshold=1e-4)

for e in range(epoch):
  labels,pred,loss=run_model(model,train_dataloader,optimizer)
  
  # calculate acc, f1 score, recall ......
  print(loss)



0.8670488479323784
0.2687784488530867
0.16117435325678
0.11259697663530888
0.09124504238301638
0.07262077748727974
0.061590728874229596
0.05545153024944203
0.05095623431328773
0.045827946594519824
0.04278914659446143
0.04051314480502065
0.03756986576649263
0.03480154022199093
0.03421243987500032
0.033969776289936936
0.032344872248433965
0.030693142195884763
0.030227340200315953
0.029524272861912663
